In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from scipy.special import logit, expit
import pickle

In [10]:
glist = ['air_u67', 'airx_u67',
           'preresnet_u67','preresnet_u101',
           'resnet_w65', 'se_resnext_u29'
           ]

In [11]:
nfold = 4

oof_dir = '../../Human_Protein_Atlas/metafeatures/oof/'

In [12]:

PATH_TO_TRAIN = '../../Human_Protein_Atlas/input/train/'
PATH_TO_TEST = '../../Human_Protein_Atlas/input/test/'
PATH_TO_TARGET = '../../Human_Protein_Atlas/input/train.csv'
PATH_TO_SUB = '../../Human_Protein_Atlas/input/sample_submission.csv'

In [13]:
sub = pd.read_csv(PATH_TO_SUB)

In [14]:
sub.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0


In [15]:
df = pd.read_csv(PATH_TO_TARGET)
df.set_index('Id',inplace=True)
print(df.head())
print(df.shape)

                                       Target
Id                                           
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18
(31072, 1)


In [16]:
file_list = list(df.index.values)

ss = pd.read_csv(PATH_TO_SUB)
ss.set_index('Id',inplace=True)
print(ss.head())
print(ss.shape)

                                      Predicted
Id                                             
00008af0-bad0-11e8-b2b8-ac1f6b6435d0          0
0000a892-bacf-11e8-b2b8-ac1f6b6435d0          0
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0          0
0008baca-bad7-11e8-b2b9-ac1f6b6435d0          0
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0          0
(11702, 1)


In [17]:
oof = df.copy()
# print(oof.head())
# print(oof.shape)
eps = 1e-6

In [18]:
glist = ['air_u67', 'airx_u67',
           'preresnet_u67','preresnet_u101',
           'resnet_w65', 'se_resnext_u29'
           ]


oof = df.copy()
# print(oof.head())
# print(oof.shape)
eps = 1e-6

for k,m in enumerate(glist):
    # print(m),
    first = True
    for fold in range(nfold):
        idf, pooff, yooff = pickle.load(open(oof_dir+m+'_'+str(fold)+'.pkl','rb'))
        # change to probability scale
        pooff = expit(pooff)
        

        if first:
            id = idf
            poof = pooff.copy()
            yoof = yooff.copy()
            first = False
        else:
            id = id + idf
            poof = np.concatenate((poof,pooff))
            yoof = np.concatenate((yoof,yooff))
    
    mdf = pd.DataFrame({'Id':id})
    # print(poof.shape)
    if k==0: mdf['y'] = [yoof[i] for i in range(yoof.shape[0])]
    mdf[m] = [poof[i] for i in range(poof.shape[0])]
    mdf.set_index('Id',inplace=True)
    oof = oof.join(mdf)

In [19]:
oof.shape

(31072, 8)

In [20]:
oof.head()

,Target,y,air_u67,airx_u67,preresnet_u67,preresnet_u101,resnet_w65,se_resnext_u29
Id,,,,,,,,
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.9552974]], [[0.011893719]], [[0.02456743]...","[[[0.978872]], [[0.018665675]], [[0.018864695]...","[[[0.9008]], [[0.04977778]], [[0.06152666]], [...","[[[0.9928462]], [[0.0033091686]], [[0.02205755...","[[[0.9949687]], [[0.008769571]], [[0.05128505]...","[[[0.99430543]], [[0.016006052]], [[0.02002830..."
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[[[0.97475284]], [[0.9937784]], [[0.37172255]]...","[[[0.9416815]], [[0.9996081]], [[0.13789588]],...","[[[0.7416709]], [[0.99948597]], [[0.072097324]...","[[[0.8883914]], [[0.99836713]], [[0.21986316]]...","[[[0.9355388]], [[0.99766314]], [[0.25203508]]...","[[[0.7101919]], [[0.969549]], [[0.27546024]], ..."
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[[[0.9680168]], [[0.0082058655]], [[0.29154035...","[[[0.92289203]], [[0.0016525302]], [[0.3881246...","[[[0.9560007]], [[0.0061894692]], [[0.3604504]...","[[[0.93580484]], [[0.037184846]], [[0.48354197...","[[[0.89877224]], [[0.007357184]], [[0.47415993...","[[[0.9792124]], [[0.017325625]], [[0.5496751]]..."
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.9572456]], [[0.99069935]], [[0.05033453]]...","[[[0.82916695]], [[0.9932394]], [[0.038969096]...","[[[0.31696457]], [[0.99046564]], [[0.003620990...","[[[0.90816665]], [[0.998226]], [[0.04174795]],...","[[[0.9247875]], [[0.9722527]], [[0.03123361]],...","[[[0.846639]], [[0.99171513]], [[0.06439011]],..."
001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.184256]], [[0.0006398597]], [[0.014315954...","[[[0.4236294]], [[0.0005882182]], [[0.00424228...","[[[0.037844375]], [[0.00064213324]], [[0.00495...","[[[0.6757472]], [[0.0011689868]], [[0.01058629...","[[[0.12146013]], [[0.000605252]], [[0.04767052...","[[[0.5738917]], [[0.0033817638]], [[0.01807783..."


In [21]:
y = np.array(oof.y.values.tolist())

In [22]:
y.shape

(31072, 28)

In [23]:

ymean = np.mean(y,axis=0)
print(ymean)

[4.1468203e-01 4.0357880e-02 1.1653579e-01 5.0238155e-02 5.9796602e-02
 8.0876671e-02 3.2440782e-02 9.0821318e-02 1.7057158e-03 1.4482492e-03
 9.0113288e-04 3.5176363e-02 2.2142122e-02 1.7282441e-02 3.4307417e-02
 6.7584962e-04 1.7057158e-02 6.7584962e-03 2.9029351e-02 4.7695674e-02
 5.5355304e-03 1.2155639e-01 2.5811020e-02 9.5423535e-02 1.0363027e-02
 2.6480433e-01 1.0556128e-02 3.5401649e-04]


In [24]:
thresholds =np.transpose(np.array([list(np.linspace(0.01,0.99, 99))]*28))

In [25]:
air_u67 = np.array(oof.air_u67.values.tolist())

air_u67 = air_u67.reshape((31072, 28))
Big_Tensor = (air_u67[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7108776996492326

In [26]:
airx_u67 = np.array(oof.airx_u67.values.tolist())

airx_u67 = airx_u67.reshape((31072, 28))
Big_Tensor = (airx_u67[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
np.max(Final_Tensor, axis=0).mean()

0.7141724152027961

In [27]:
preresnet_u67 = np.array(oof.preresnet_u67.values.tolist())
preresnet_u101 = np.array(oof.preresnet_u101.values.tolist())
preresnet_u67 = preresnet_u67.reshape((31072, 28))
preresnet_u101 = preresnet_u101.reshape((31072, 28))

In [28]:
Big_Tensor = (preresnet_u67[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
Final_Tensor = np.array(Final_Tensor)
np.max(Final_Tensor, axis=0).mean()

0.7092749679908568

In [29]:
Big_Tensor = (preresnet_u101[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
Final_Tensor = np.array(Final_Tensor)
np.max(Final_Tensor, axis=0).mean()

0.7056871990007961

In [30]:
se_resnext_u29 = np.array(oof.se_resnext_u29.values.tolist())
resnet_w65 = np.array(oof.resnet_w65.values.tolist())
se_resnext_u29 = se_resnext_u29.reshape((31072, 28))
resnet_w65 = resnet_w65.reshape((31072, 28))

In [31]:
Big_Tensor = (se_resnext_u29[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
Final_Tensor = np.array(Final_Tensor)
np.max(Final_Tensor, axis=0).mean()

0.7015426593208052

In [32]:
Big_Tensor = (resnet_w65[:,np.newaxis] > thresholds)*1.
Final_Tensor = [f1_score(y, Big_Tensor[:,x,:], average=None) for x in range(99)]
Final_Tensor = np.array(Final_Tensor)
np.max(Final_Tensor, axis=0).mean()

0.7088338470137628

In [33]:
thresholds_0 = np.linspace(0.01, 0.99, 99)

In [34]:
pred0 = np.load('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/pred0.npy')
pred1 = np.load('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/pred1.npy')
pred2 = np.load('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/pred2.npy')
pred3 = np.load('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/pred3.npy')

In [36]:
pred1.shape

(11702, 28)

In [37]:
thresholds_0 = np.linspace(0.01, 0.99, 99)
train_six = np.hstack((air_u67, airx_u67, preresnet_u67, preresnet_u101, resnet_w65, se_resnext_u29))
train_six.shape

In [38]:
train_six.shape

(31072, 168)

In [39]:
cluster4x_folds = pd.read_csv('../Russ/cluster4x_folds.csv')

In [40]:
cluster4x_folds.head()

,Id,cluster4
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,1
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,1
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,0
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,3


In [41]:
np.unique(cluster4x_folds.cluster4.values, return_counts=True)

(array([0, 1, 2, 3]), array([ 9970, 15381, 13030, 18620]))

In [42]:
oof_folds = oof.merge(cluster4x_folds, on='Id')

In [43]:
oof_folds.head()

,Id,Target,y,air_u67,airx_u67,preresnet_u67,preresnet_u101,resnet_w65,se_resnext_u29,cluster4
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.9552974]], [[0.011893719]], [[0.02456743]...","[[[0.978872]], [[0.018665675]], [[0.018864695]...","[[[0.9008]], [[0.04977778]], [[0.06152666]], [...","[[[0.9928462]], [[0.0033091686]], [[0.02205755...","[[[0.9949687]], [[0.008769571]], [[0.05128505]...","[[[0.99430543]], [[0.016006052]], [[0.02002830...",0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[[[0.97475284]], [[0.9937784]], [[0.37172255]]...","[[[0.9416815]], [[0.9996081]], [[0.13789588]],...","[[[0.7416709]], [[0.99948597]], [[0.072097324]...","[[[0.8883914]], [[0.99836713]], [[0.21986316]]...","[[[0.9355388]], [[0.99766314]], [[0.25203508]]...","[[[0.7101919]], [[0.969549]], [[0.27546024]], ...",1
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[[[0.9680168]], [[0.0082058655]], [[0.29154035...","[[[0.92289203]], [[0.0016525302]], [[0.3881246...","[[[0.9560007]], [[0.0061894692]], [[0.3604504]...","[[[0.93580484]], [[0.037184846]], [[0.48354197...","[[[0.89877224]], [[0.007357184]], [[0.47415993...","[[[0.9792124]], [[0.017325625]], [[0.5496751]]...",1
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.9572456]], [[0.99069935]], [[0.05033453]]...","[[[0.82916695]], [[0.9932394]], [[0.038969096]...","[[[0.31696457]], [[0.99046564]], [[0.003620990...","[[[0.90816665]], [[0.998226]], [[0.04174795]],...","[[[0.9247875]], [[0.9722527]], [[0.03123361]],...","[[[0.846639]], [[0.99171513]], [[0.06439011]],...",0
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[[0.184256]], [[0.0006398597]], [[0.014315954...","[[[0.4236294]], [[0.0005882182]], [[0.00424228...","[[[0.037844375]], [[0.00064213324]], [[0.00495...","[[[0.6757472]], [[0.0011689868]], [[0.01058629...","[[[0.12146013]], [[0.000605252]], [[0.04767052...","[[[0.5738917]], [[0.0033817638]], [[0.01807783...",3


In [44]:
cluster4 = oof_folds.cluster4.values

In [45]:
y[:,0][np.where(cluster4 == 0)[0]]

array([1., 0., 1., ..., 1., 0., 0.], dtype=float32)

In [46]:
from sklearn.linear_model import Ridge

In [48]:
new_oof = np.zeros(train_six.shape[0])
for i in range(4):
    val_idx = np.where(cluster4 == i)[0]
    trn_idx = np.where(cluster4 != i)[0]
    train_x, val_x = train_six[trn_idx, :], train_six[val_idx, :]
    train_y, val_y = y[:,0][trn_idx], y[:,0][val_idx]
    clf = Ridge(alpha=0)
    clf.fit(train_x, train_y)
    oof_preds = clf.predict(val_x)
    new_oof[val_idx] = oof_preds
    


In [49]:
new_oof.max()

1.1296746730804443

In [50]:
val_x.shape

(10771, 168)

In [51]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler

In [52]:
scaler = MinMaxScaler()
new_oof = scaler.fit_transform(new_oof.reshape(-1,1))

In [53]:
new_oof.max()

1.0

In [54]:
f1_score(y[:,0], (new_oof > 0.68)*1.)

0.8423565709248513

In [56]:
f1_score(y[:,0], (train_six[:,0] > 0.88)*1.)

0.8341106434611155

In [58]:
scores = [f1_score(y[:,0], (new_oof > th)*1.,) for th in thresholds_0]

In [59]:
np.max(scores)

0.8423565709248513

In [116]:
scores = [f1_score(y[:,0], (train_five[:,4*28] > th)*1.,) for th in threshold_0]
np.max(scores)

0.8280603193376701

In [60]:
from tqdm import tqdm

In [62]:
all_columns = set(range(168))

all_good_features = []
all_best_scores = []

for ii in tqdm(range(28)):
    print("Fitting feature", ii)
    first_six =  set(np.array(range(6))*(28)+ii)
    the_rest = np.array(list(all_columns - first_six))
    the_list = list(np.sort(list(first_six)))+list(the_rest)
    best_score = 0
    good_features = []
    for feature in the_list:
        new_oof = np.zeros(train_six.shape[0])
        temp_good_features = good_features+[feature]
        for i in range(4):
            val_idx = np.where(cluster4 == i)[0]
            trn_idx = np.where(cluster4 != i)[0]
            train_x, val_x = train_six[:,temp_good_features][trn_idx, :], train_six[:,temp_good_features][val_idx, :]
            train_y, val_y = y[:,ii][trn_idx], y[:,ii][val_idx]
            clf = Ridge(alpha=0)
            clf.fit(train_x, train_y)
            oof_preds = clf.predict(val_x)
            new_oof[val_idx] = oof_preds
            
        scaler = MinMaxScaler()
        new_oof = scaler.fit_transform(new_oof.reshape(-1,1))
        scores = [f1_score(y[:,ii], (new_oof > th)*1.,) for th in thresholds_0]
        new_score = np.max(scores)
        if new_score > best_score:
            print(feature)
            print(new_score)
            best_score = new_score
            good_features.append(feature)
            
    all_best_scores.append(best_score)
            
    all_good_features.append(good_features)
        

  0%|          | 0/28 [00:00<?, ?it/s]

Fitting feature 0
0
0.8360929557007989
28
0.8416567871962062
56
0.8438464931627702
84
0.8460675837320574
140
0.8489261547513974
3
0.8490343505823381
8
0.8491101367036369
9
0.8492262343404569
18
0.8492848475934845
30
0.849408033046878
41
0.8496312684365782
50
0.8497237569060774
63
0.8497463056923817
64
0.8497990190655308
67
0.8498063087991146
68
0.8498625659312087
78
0.8499219504943135
81
0.8500166771671053
82
0.8500592943966795
83
0.850090797909795
121
0.8501445193804196
155
0.8501538119417368


  4%|▎         | 1/28 [02:21<1:03:48, 141.79s/it]

Fitting feature 1
1
0.834178696839641
29
0.8453855373551739
57
0.8505935325419566
85
0.8531965272296764
113
0.8542673107890499
141
0.8618342010412495
0
0.8631163708086785
6
0.8639240506329113
11
0.8641584158415841
23
0.8646080760095013
109
0.8648434403487911
116
0.8651863600317207


  7%|▋         | 2/28 [04:23<58:49, 135.75s/it]  

...

 93%|█████████▎| 26/28 [1:10:43<04:18, 129.44s/it]

Fitting feature 26
26
0.528358208955224
54
0.5576923076923077
82
0.5774647887323944
110
0.5846599131693198
5
0.5858585858585859
25
0.5863570391872279
30
0.5884057971014492
47
0.5892597968069666


 96%|█████████▋| 27/28 [1:12:42<02:06, 126.14s/it]

Fitting feature 27
27
0.7200000000000001
55
0.7826086956521738
83
0.8181818181818182


100%|██████████| 28/28 [1:14:36<00:00, 122.51s/it]


In [64]:
np.mean(all_best_scores)

0.7498756304716229

In [66]:
all_good_features_array = np.array(all_good_features)

In [67]:
all_good_features_array

array([list([0, 28, 56, 84, 140, 3, 8, 9, 18, 30, 41, 50, 63, 64, 67, 68, 78, 81, 82, 83, 121, 155]),
       list([1, 29, 57, 85, 113, 141, 0, 6, 11, 23, 109, 116]),
       list([2, 30, 58, 114, 0, 3, 4, 7, 12, 19, 21, 22, 27, 40, 41, 46, 47, 74, 75, 82, 83, 111, 154]),
       list([3, 31, 59, 87, 115, 0, 4, 5, 6, 11, 12, 18, 32, 39, 124]),
       list([4, 32, 60, 116, 0, 1, 6, 46, 50, 103, 140, 142, 143]),
       list([5, 33, 89, 117, 3, 6, 7, 11, 25, 26, 31, 40, 41, 55, 57, 58, 62, 63, 67, 68, 70, 77, 79, 87, 95, 96, 98, 108, 111, 115, 120, 127, 143, 158]),
       list([6, 34, 62, 118, 146, 2, 5, 10, 16, 21, 25, 31, 48, 50, 56]),
       list([7, 35, 63, 91, 119, 147, 1, 2, 23, 25, 28, 29, 30, 32, 37, 46, 47, 53, 69, 84, 85, 87, 90, 92, 124, 138]),
       list([8, 36, 92]), list([9, 37, 93, 121]),
       list([10, 38, 66, 94, 122]),
       list([11, 39, 67, 95, 123, 0, 4, 6, 7, 22, 84, 112, 113, 115, 129]),
       list([12, 40, 68, 124, 152, 13, 21, 22, 41, 58, 61]),
       list([13, 

In [68]:
np.save('all_good_features_2', all_good_features)

In [69]:
all_good_features_2 = np.load('all_good_features_2.npy')

In [70]:
all_good_features_2

array([list([0, 28, 56, 84, 140, 3, 8, 9, 18, 30, 41, 50, 63, 64, 67, 68, 78, 81, 82, 83, 121, 155]),
       list([1, 29, 57, 85, 113, 141, 0, 6, 11, 23, 109, 116]),
       list([2, 30, 58, 114, 0, 3, 4, 7, 12, 19, 21, 22, 27, 40, 41, 46, 47, 74, 75, 82, 83, 111, 154]),
       list([3, 31, 59, 87, 115, 0, 4, 5, 6, 11, 12, 18, 32, 39, 124]),
       list([4, 32, 60, 116, 0, 1, 6, 46, 50, 103, 140, 142, 143]),
       list([5, 33, 89, 117, 3, 6, 7, 11, 25, 26, 31, 40, 41, 55, 57, 58, 62, 63, 67, 68, 70, 77, 79, 87, 95, 96, 98, 108, 111, 115, 120, 127, 143, 158]),
       list([6, 34, 62, 118, 146, 2, 5, 10, 16, 21, 25, 31, 48, 50, 56]),
       list([7, 35, 63, 91, 119, 147, 1, 2, 23, 25, 28, 29, 30, 32, 37, 46, 47, 53, 69, 84, 85, 87, 90, 92, 124, 138]),
       list([8, 36, 92]), list([9, 37, 93, 121]),
       list([10, 38, 66, 94, 122]),
       list([11, 39, 67, 95, 123, 0, 4, 6, 7, 22, 84, 112, 113, 115, 129]),
       list([12, 40, 68, 124, 152, 13, 21, 22, 41, 58, 61]),
       list([13, 

In [72]:
Christof_oof_preds = pd.read_csv('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/oof_pred.csv')

In [73]:
Christof_oof_preds.head()

,Id,Target,fold,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,...,pred_18,pred_19,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0,0,0.953081,0.002805,0.045376,0.001540,0.003110,0.069791,0.000767,...,0.970529,0.481568,0.000034,0.122002,0.004993,0.015003,0.000105,0.432573,0.000032,6.785499e-07
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0,1,0.769686,0.999566,0.425338,0.012970,0.000898,0.040802,0.002442,...,0.001096,0.006145,0.000011,0.021171,0.003723,0.005379,0.000575,0.139700,0.000336,2.664577e-06
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5,1,0.920565,0.010179,0.531457,0.423629,0.024148,0.918234,0.012125,...,0.267045,0.300161,0.000151,0.134051,0.087772,0.030957,0.000593,0.482627,0.001335,1.798913e-05
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1,0,0.758010,0.998355,0.013991,0.001843,0.012320,0.005001,0.321245,...,0.000994,0.003252,0.000012,0.108415,0.011570,0.068579,0.000002,0.652738,0.000047,4.592995e-06
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18,3,0.505837,0.000687,0.034368,0.007865,0.002112,0.010435,0.001773,...,0.993149,0.464579,0.000035,0.010500,0.002290,0.002902,0.005780,0.261669,0.000003,4.012958e-07


In [74]:
Christof_oof_preds.shape

(31072, 31)

In [75]:
train_six.shape

(31072, 168)

In [79]:
Christof_oof_preds = Christof_oof_preds[Christof_oof_preds.columns[3:]].values

In [80]:
Christof_oof_preds.shape

(31072, 28)

In [81]:
train_seven = np.hstack((train_six, Christof_oof_preds))

In [82]:
train_seven.shape

(31072, 196)

In [83]:
all_columns = set(range(196))

all_good_features = []
all_best_scores = []

for ii in tqdm(range(28)):
    print("Fitting feature", ii)
    first_seven =  set(np.array(range(7))*(28)+ii)
    the_rest = np.array(list(all_columns - first_seven))
    the_list = list(np.sort(list(first_seven)))+list(the_rest)
    best_score = 0
    good_features = []
    for feature in the_list:
        new_oof = np.zeros(train_seven.shape[0])
        temp_good_features = good_features+[feature]
        for i in range(4):
            val_idx = np.where(cluster4 == i)[0]
            trn_idx = np.where(cluster4 != i)[0]
            train_x, val_x = train_seven[:,temp_good_features][trn_idx, :], train_seven[:,temp_good_features][val_idx, :]
            train_y, val_y = y[:,ii][trn_idx], y[:,ii][val_idx]
            clf = Ridge(alpha=0)
            clf.fit(train_x, train_y)
            oof_preds = clf.predict(val_x)
            new_oof[val_idx] = oof_preds
            
        scaler = MinMaxScaler()
        new_oof = scaler.fit_transform(new_oof.reshape(-1,1))
        scores = [f1_score(y[:,ii], (new_oof > th)*1.,) for th in thresholds_0]
        new_score = np.max(scores)
        if new_score > best_score:
            print(feature)
            print(new_score)
            best_score = new_score
            good_features.append(feature)
            
    all_best_scores.append(best_score)
            
    all_good_features.append(good_features)
        

  0%|          | 0/28 [00:00<?, ?it/s]

Fitting feature 0
0
0.8360929557007989
28
0.8416567871962062
56
0.8438464931627702
84
0.8460675837320574
140
0.8489261547513974
168
0.8495338558110166
3
0.8497697905836922
9
0.8497790321981653
12
0.849866745632218
13
0.8499462106317468
15
0.8500741839762611
20
0.8502111267501297


  4%|▎         | 1/28 [02:58<1:20:07, 178.07s/it]

Fitting feature 1
1
0.834178696839641
29
0.8453855373551739
57
0.8505935325419566
85
0.8531965272296764
113
0.8542673107890499
141
0.8618342010412495
169
0.8651199370821864
0
0.8653093187157401
4
0.8656482569526047
5
0.8661971830985916
6
0.8665362035225048
8
0.8666928824223359
11
0.8667711598746083
19
0.8672150411280845
45
0.8675548589341693
146
0.8679984332158245


  7%|▋         | 2/28 [05:38<1:14:53, 172.82s/it]

...

 93%|█████████▎| 26/28 [1:07:45<05:24, 162.16s/it]

Fitting feature 26
26
0.528358208955224
54
0.5576923076923077
82
0.5774647887323944
110
0.5846599131693198
5
0.5858585858585859
25
0.5863570391872279
30
0.5884057971014492
47
0.5892597968069666


 96%|█████████▋| 27/28 [1:10:13<02:37, 157.67s/it]

Fitting feature 27
27
0.7200000000000001
55
0.7826086956521738
83
0.8181818181818182


100%|██████████| 28/28 [1:12:33<00:00, 152.41s/it]


In [84]:
np.mean(all_best_scores)

0.7523193382946214

In [85]:
np.save('all_good_features_3', all_good_features)

In [86]:
sub_dir = '../../Human_Protein_Atlas/metafeatures/sub/'

In [97]:
idf, pooff, yooff = pickle.load(open(sub_dir+glist[0]+'_'+str(0)+'_mm.pkl','rb'))

In [100]:
len(idf)

11702

In [102]:
oof.shape

(31072, 8)

In [107]:
yooff = yooff.reshape((11702, 28))

In [104]:
sub = pd.DataFrame({'Id':idf})
mdf[m] = [poof[i] for i in range(poof.shape[0])]
sub[glist[0]] = yooff
sub.head()

AssertionError: Shape of new values must be compatible with manager shape

In [109]:
train_seven

array([[9.55297410e-01, 1.18937194e-02, 2.45674308e-02, ...,
        4.32572931e-01, 3.18431885e-05, 6.78549895e-07],
       [9.74752843e-01, 9.93778408e-01, 3.71722549e-01, ...,
        1.39700353e-01, 3.35894380e-04, 2.66457664e-06],
       [9.68016803e-01, 8.20586551e-03, 2.91540354e-01, ...,
        4.82627362e-01, 1.33526605e-03, 1.79891267e-05],
       ...,
       [5.15302159e-02, 1.68831944e-02, 9.27164927e-02, ...,
        6.66059852e-01, 2.07813628e-05, 1.10266128e-05],
       [3.96215528e-01, 1.85226798e-02, 9.03444052e-01, ...,
        9.90998745e-01, 7.64248520e-03, 1.78813445e-06],
       [9.93071616e-01, 2.34166771e-01, 9.96309578e-01, ...,
        6.13405406e-01, 2.90418714e-02, 2.25477779e-05]])

In [ ]:

sub = df.copy()
# print(oof.head())
# print(oof.shape)
eps = 1e-6

for k,m in enumerate(glist):
    # print(m),
    first = True
    for fold in range(nfold):
        idf, pooff, yooff = pickle.load(open(sub_dir+m+'_'+str(fold)+'_mm.pkl','rb'))
        # change to probability scale
        pooff = expit(pooff)
        

        if first:
            id = idf
            poof = pooff.copy()
            yoof = yooff.copy()
            first = False
        else:
            id = id + idf
            poof = np.concatenate((poof,pooff))
            yoof = np.concatenate((yoof,yooff))
    
    mdf = pd.DataFrame({'Id':id})
    # print(poof.shape)
    if k==0: mdf['y'] = [yoof[i] for i in range(yoof.shape[0])]
    mdf[m] = [poof[i] for i in range(poof.shape[0])]
    mdf.set_index('Id',inplace=True)
    oof = oof.join(mdf)

In [121]:
yoof

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [130]:
folds = []

for fold in range(nfold):
    preds = []
    for k,m in enumerate(glist):
        idf, pooff, yooff = pickle.load(open(sub_dir+m+'_'+str(fold)+'_mm.pkl','rb'))
        pooff = pooff.reshape((11702, 28))
        preds.append(pooff)
    christoff_pred = np.load('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/pred'+str(fold)+'.npy')
    preds.append(christoff_pred)
    folds.append(preds)

In [131]:
np.hstack(folds[0]).shape

(11702, 196)

In [128]:
christoff_pred_0 = np.load('../../Human_Protein_Atlas/metafeatures/oof/GAPNet_14_ext/pred0.npy')

In [129]:
christoff_pred_0.shape

(11702, 28)

In [138]:
np.hstack(folds[0])[:,all_good_features[0]].shape

(11702, 12)

In [133]:
len(all_good_features)

28

In [134]:
from sklearn.linear_model import LogisticRegression

In [136]:
clf = LogisticRegression()

In [ ]:
clf.pred

In [214]:
best_scores = []
best_thresholds = []
total_preds = []
total_preds_all_folds = []
oofs = []
for ii in tqdm(range(28)):
    print("Fitting feature", ii)
    new_oof = np.zeros(train_seven.shape[0])
    new_oof = new_oof.reshape(-1,1)
    feature_preds = 0
    feature_preds_list = []
    for i in range(4):
        val_idx = np.where(cluster4 == i)[0]
        trn_idx = np.where(cluster4 != i)[0]
        train_x, val_x = train_seven[:,all_good_features[ii]][trn_idx, :], train_seven[:,all_good_features[ii]][val_idx, :]
        train_y, val_y = y[:,ii][trn_idx], y[:,ii][val_idx]
        clf = LogisticRegression(C=0.2)
        clf.fit(train_x, train_y)
        oof_preds = clf.predict_proba(val_x)[:,1]
        new_oof[val_idx] = oof_preds.reshape(-1,1)
        feature_preds += 0.25*clf.predict_proba(np.hstack(folds[i])[:,all_good_features[ii]])[:,1]
        feature_preds_list.append(clf.predict_proba(np.hstack(folds[i])[:,all_good_features[ii]])[:,1])
            
    oofs.append(new_oof)

    #scores = [f1_score(y[:,ii], (new_oof > th)*1.,) for th in thresholds_0]
    #new_score = np.max(scores)
    #best_scores.append(best_score)
    #best_thresholds.append((np.argmax(scores)+1)/100.)
    
    total_preds.append(feature_preds)
    total_preds_all_folds.append(feature_preds_list)

        


  0%|          | 0/28 [00:00<?, ?it/s]

Fitting feature 0


/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)

  4%|▎         | 1/28 [00:01<00:40,  1.50s/it]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433

Fitting feature 1


/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)

  7%|▋         | 2/28 [00:01<00:30,  1.17s/it]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

Fitting feature 25


/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)

 93%|█████████▎| 26/28 [00:11<00:01,  1.99it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:43

Fitting feature 26


 96%|█████████▋| 27/28 [00:12<00:00,  2.42it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)

100%|██████████| 28/28 [00:12<00:00,  2.99it/s]


Fitting feature 27


In [191]:
oofs[0]

array([[0.82185963],
       [0.58871823],
       [0.84771541],
       ...,
       [0.00837861],
       [0.03938513],
       [0.85734962]])

In [158]:
oofs[1]

array([[7.47655741e-04],
       [8.73129515e-01],
       [8.33156465e-04],
       ...,
       [2.56638777e-03],
       [2.55098265e-03],
       [4.59512389e-03]])

In [159]:
len(total_preds)

28

In [161]:
total_preds[0]

array([9.55019952e-05, 3.35504022e-04, 9.99999594e-01, ...,
       1.13655942e-03, 9.79716737e-01, 9.95445614e-01])

In [166]:
train_seven[:,0]

array([0.95529741, 0.97475284, 0.9680168 , ..., 0.05153022, 0.39621553,
       0.99307162])

In [169]:
scores = [f1_score(y[:,0], (oofs[0].flatten() > th)*1.,) for th in thresholds_0]

/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [170]:
np.max(scores)

0.8493475682087782

In [205]:
best_scores = []
best_thresholds = []
for ii in tqdm(range(28)):
    scores = [f1_score(y[:,ii], (oofs[ii].flatten() > th)*1.,) for th in thresholds_0]
    best_scores.append(np.max(scores))
    best_thresholds.append((np.argmax(scores)+1)/100.)


  0%|          | 0/28 [00:00<?, ?it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)

100%|██████████| 28/28 [00:20<00:00,  1.41it/s]


In [174]:
np.mean(best_scores)

0.7544497944033335

In [183]:
np.mean(best_scores)

0.7536573651894838

In [186]:
np.mean(best_scores)

0.7547762635387866

In [189]:
np.mean(best_scores)

0.7499543011234424

In [193]:
np.mean(best_scores)

0.7547412880140296

In [196]:
np.mean(best_scores)

0.7522404917924858

In [200]:
np.mean(best_scores)

0.7546804953838604

In [203]:
np.mean(best_scores)

0.7529747904530862

In [206]:
np.mean(best_scores)

0.7547762635387866

In [207]:
best_thresholds

[0.58,
 0.6,
 0.62,
 0.42,
 0.51,
 0.51,
 0.3,
 0.61,
 0.04,
 0.05,
 0.05,
 0.54,
 0.41,
 0.5,
 0.59,
 0.01,
 0.31,
 0.26,
 0.37,
 0.39,
 0.41,
 0.43,
 0.41,
 0.43,
 0.27,
 0.42,
 0.27,
 0.03]

In [176]:
thresholds_0[0]

0.01

In [178]:
np.max(scores)

0.8421052631578948

In [179]:
np.argmax(scores)

26

In [180]:
scores

[0.5714285714285714,
 0.5925925925925926,
 0.64,
 0.6666666666666667,
 0.6666666666666667,
 0.6666666666666667,
 0.6666666666666667,
 0.6956521739130435,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.7272727272727273,
 0.761904761904762,
 0.761904761904762,
 0.761904761904762,
 0.761904761904762,
 0.761904761904762,
 0.7999999999999999,
 0.7999999999999999,
 0.7999999999999999,
 0.7999999999999999,
 0.7999999999999999,
 0.7999999999999999,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.8421052631578948,
 0.7777777777777778,
 0.7777777777777778,
 0.7058823529411764,
 0.7058823529411764,
 0.5333333333333333,
 0.53333333

In [208]:
thresholds_1000 = np.linspace(0.001, 0.999, 999)

In [210]:
thresholds_1000.shape

(999,)

In [211]:
best_scores = []
best_thresholds = []
for ii in tqdm(range(28)):
    scores = [f1_score(y[:,ii], (oofs[ii].flatten() > th)*1.,) for th in thresholds_1000]
    best_scores.append(np.max(scores))
    best_thresholds.append((np.argmax(scores)+1)/1000.)


  0%|          | 0/28 [00:00<?, ?it/s]/Users/btunguz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)

100%|██████████| 28/28 [03:20<00:00,  7.29s/it]


In [212]:
np.mean(best_scores)

0.7552140501350012

In [213]:
best_thresholds

[0.581,
 0.606,
 0.626,
 0.415,
 0.522,
 0.51,
 0.352,
 0.613,
 0.036,
 0.045,
 0.048,
 0.539,
 0.392,
 0.501,
 0.586,
 0.01,
 0.312,
 0.26,
 0.37,
 0.391,
 0.438,
 0.428,
 0.41,
 0.427,
 0.27,
 0.418,
 0.283,
 0.029]

In [216]:
len(oofs)

28

In [220]:
oofs_ar = np.array(np.transpose(oofs))

In [222]:
oofs_ar = oofs_ar.reshape((31072, 28))

In [223]:
oof_df = pd.DataFrame(data=oofs_ar)

In [224]:
oof_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.819872,0.001255,0.010088,0.004730,0.004004,0.072005,0.003875,0.005704,0.001824,0.001772,...,0.017457,0.017645,0.002358,0.011077,0.003908,0.007014,0.003239,0.053142,0.003605,0.001558
1,0.593233,0.861980,0.026842,0.005026,0.003742,0.013390,0.010246,0.253743,0.002087,0.002068,...,0.002716,0.005479,0.002469,0.006166,0.004643,0.005053,0.003074,0.012185,0.005409,0.001738
2,0.842677,0.001378,0.059814,0.155937,0.005004,0.704619,0.002630,0.016034,0.002102,0.002065,...,0.002074,0.010153,0.007015,0.007338,0.003464,0.008193,0.001922,0.032244,0.003139,0.001744
3,0.459891,0.896107,0.009642,0.005258,0.004286,0.011575,0.013448,0.004831,0.001820,0.001767,...,0.004495,0.004501,0.002326,0.007614,0.003822,0.006508,0.003195,0.241150,0.004394,0.001550
4,0.059189,0.003724,0.007134,0.005194,0.007420,0.011877,0.009707,0.010360,0.002279,0.002170,...,0.788075,0.014129,0.002708,0.009455,0.005474,0.003048,0.004225,0.010782,0.005728,0.001829


In [225]:
oofs[0].flatten()

array([0.81987165, 0.5932327 , 0.84267659, ..., 0.00899772, 0.04146459,
       0.85461374])

In [230]:
oof_df['Id'] = cluster4x_folds['Id']
oof_df['cluster4'] = cluster4x_folds['cluster4']

In [231]:
oof_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,Id,cluster4
0,0.819872,0.001255,0.010088,0.004730,0.004004,0.072005,0.003875,0.005704,0.001824,0.001772,...,0.002358,0.011077,0.003908,0.007014,0.003239,0.053142,0.003605,0.001558,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,0
1,0.593233,0.861980,0.026842,0.005026,0.003742,0.013390,0.010246,0.253743,0.002087,0.002068,...,0.002469,0.006166,0.004643,0.005053,0.003074,0.012185,0.005409,0.001738,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,1
2,0.842677,0.001378,0.059814,0.155937,0.005004,0.704619,0.002630,0.016034,0.002102,0.002065,...,0.007015,0.007338,0.003464,0.008193,0.001922,0.032244,0.003139,0.001744,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,1
3,0.459891,0.896107,0.009642,0.005258,0.004286,0.011575,0.013448,0.004831,0.001820,0.001767,...,0.002326,0.007614,0.003822,0.006508,0.003195,0.241150,0.004394,0.001550,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,0
4,0.059189,0.003724,0.007134,0.005194,0.007420,0.011877,0.009707,0.010360,0.002279,0.002170,...,0.002708,0.009455,0.005474,0.003048,0.004225,0.010782,0.005728,0.001829,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,3


In [240]:
total_preds_all_folds[17][3].shape

(11702,)

In [245]:
all_preds = []

for i in range(4):
    temp_preds = []
    for j in range(28):
        temp_preds.append(total_preds_all_folds[j][i])
    all_preds.append(pd.DataFrame(data=np.transpose(np.array(temp_preds))))
    

In [246]:
len(all_preds)

4

In [250]:
all_preds[3].head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.001263,3.099695e-07,1.000000e+00,8.739093e-14,2.144667e-17,4.261845e-11,2.081191e-13,2.346268e-12,1.178048e-25,6.566304e-27,...,1.055553e-18,3.793562e-10,3.345323e-24,1.896347e-10,4.379397e-09,1.993170e-02,2.709866e-20,1.086190e-12,1.873815e-19,3.163003e-15
1,0.000001,1.274295e-15,2.139089e-07,9.175001e-12,1.175656e-09,1.000000e+00,1.043879e-08,1.503118e-11,1.710132e-28,4.077621e-29,...,1.176809e-07,7.295626e-11,1.883266e-22,2.332208e-07,1.565213e-10,2.918335e-06,1.551856e-13,5.303397e-01,3.652265e-12,4.333532e-12
2,0.999999,1.661266e-16,3.827674e-07,1.142613e-13,3.103842e-08,1.000000e+00,1.621008e-15,1.406103e-07,1.482097e-30,1.026785e-34,...,2.075863e-12,2.278940e-06,1.758399e-28,3.071496e-03,1.805984e-13,1.725151e-09,1.309173e-19,9.996889e-01,5.885852e-23,2.778529e-13
3,1.000000,2.965215e-03,7.388267e-10,4.782987e-16,7.587603e-13,5.238220e-04,3.150887e-16,6.310522e-01,1.287754e-23,7.100084e-31,...,1.277161e-11,6.064996e-08,3.921936e-22,1.910006e-01,7.304387e-12,4.608637e-03,4.909262e-24,7.661669e-03,3.536779e-19,7.982431e-13
4,0.010350,1.852704e-07,2.395586e-06,4.135122e-10,4.424475e-15,2.046585e-07,3.036353e-14,5.648181e-02,1.943420e-16,1.221878e-30,...,1.479647e-15,1.742754e-11,1.676700e-22,4.001982e-06,9.756453e-14,9.997012e-01,1.268247e-21,2.819774e-03,7.630982e-15,3.632173e-13


In [251]:
oof_df.to_csv('../../Human_Protein_Atlas/L2_metafeatures/oof_df_LR_1.csv', index=False)

In [252]:
for i in range(4):
    all_preds[i].to_csv('../../Human_Protein_Atlas/L2_metafeatures/test_preds_LR_1_fold_'+str(i)+'.csv', index=False)